In [1]:
#1.a
import nltk
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/yiweihan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yiweihan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yiweihan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yiweihan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
data = pd.read_csv('/Users/yiweihan/Desktop/corona_fake.csv')

In [3]:
#1.b.i
data = data.dropna(axis = 0, how ='any')

In [4]:
data.index = range(len(data))
tokens = []
for i in range (data.shape[0]):
    temp = nltk.word_tokenize(data['text'][i])
    tokens.append(temp)

In [5]:
#1.b.ii
tagged =[]
for i in tokens:
    temp = nltk.pos_tag(i)
    tagged.append(temp)

In [6]:
#1.b.iii
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
wnl = WordNetLemmatizer()
lemmatize = []
for i in tagged:
    lemmas_sent= []
    for tag in i:
        wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
        lemmas_sent.append(wnl.lemmatize(tag[0], pos = wordnet_pos))
    lemmatize.append(lemmas_sent) 

In [7]:
#1.b.iv
from nltk.corpus import stopwords
filtered_words = []
for i in lemmatize:
    temp = [word for word in i if word not in stopwords.words('english')]
    filtered_words.append(temp)

In [8]:
#1.b.v
import re
 
def remove(text):
    remove_chars = '[0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
    return re.sub(remove_chars, '', text)
def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

text_clean = []
for filtered_word in filtered_words:
    for i in filtered_word:
        if len(i)<3:
            filtered_word.remove(i)
    temp1 = remove(" ".join(i for i in filtered_word))
    temp2 = remove_URL("".join(i for i in temp1))
    text_clean.append(temp2)


In [9]:
#2.a
# n-diagram is a n-item sequence, which can be phonemes, syllables, letters, words or base pairs
#from a given sample of text and speech. Based on some sample or storage of language, people can use
#this to predict or evaluate whether a sentence is reasonable. The other application can be to evaluate
#the difference between two texts. The contents of the text are operated by a sliding window with a size
#of N in bytes, forming a sequence of byte fragments with a length of N. In order to facilitate calculation,
#we only consider the front n-1 words in n-digram. N-gram’s advantage is that it is widely used to compress
#text, check for spelling errors, speed up string lookup, and document language recognition. And it is
#the most useful of automatic classification of natural languages as people can not process those huge amount
#of items by their own.

In [10]:
#2.b.
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [11]:
#2.c.
data['clean_text'] = text_clean

In [12]:
X = data["clean_text"]
Y = data["label"]
countvectorizer1 = CountVectorizer(lowercase=True, ngram_range=(1,1))
countvectorizer2 = CountVectorizer(lowercase=True, ngram_range=(1,2))
countvectorizer3 = CountVectorizer(lowercase=True, ngram_range=(1,3))
X_countvectorizer1 = countvectorizer1.fit_transform(X)
X_countvectorizer2 = countvectorizer2.fit_transform(X)
X_countvectorizer3 = countvectorizer3.fit_transform(X)

In [13]:
#2.d.
X = data["clean_text"]
Y = data["label"]
tfidfvectorizer1 = TfidfVectorizer(lowercase=True, ngram_range =(1,1))
tfidfvectorizer2 = TfidfVectorizer(lowercase=True, ngram_range=(1,2))
tfidfvectorizer3 = TfidfVectorizer(lowercase=True, ngram_range=(1,3))
X_tfidfvectorizer1 = tfidfvectorizer1.fit_transform(X)
X_tfidfvectorizer2 = tfidfvectorizer2.fit_transform(X)
X_tfidfvectorizer3 = tfidfvectorizer3.fit_transform(X)

In [14]:
#3.a.
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
CV = LogisticRegressionCV(cv=5, max_iter=1000, n_jobs = -1, random_state=265)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_countvectorizer1, Y, test_size = 0.3, random_state = 265)


c1 = CV.fit(X_train, y_train)
c1_predict = CV.predict(X_test)
C1_Accuracy = c1.score(X_test,y_test)
C1_Accuracy

0.9150943396226415

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_countvectorizer2, Y, test_size = 0.3, random_state = 265)


c2 = CV.fit(X_train, y_train)
c2_predict = CV.predict(X_test)
C2_Accuracy = c2.score(X_test,y_test)
C2_Accuracy


0.9276729559748428

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_countvectorizer3, Y, test_size = 0.3, random_state = 265)


c3 = CV.fit(X_train, y_train)
c3_predict = CV.predict(X_test)
C3_Accuracy = c3.score(X_test,y_test)
C3_Accuracy


0.9119496855345912

In [18]:
#3.b.
X_train, X_test, y_train, y_test = train_test_split(X_tfidfvectorizer1, Y, test_size = 0.3, random_state = 265)
t1 = CV.fit(X_train, y_train)
t1_predict = CV.predict(X_test)
T1_Accuracy = t1.score(X_test,y_test)
T1_Accuracy

0.940251572327044

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidfvectorizer2, Y, test_size = 0.3, random_state = 265)


t2 = CV.fit(X_train, y_train)
t2_predict = CV.predict(X_test)
T2_Accuracy = t2.score(X_test,y_test)
T2_Accuracy

0.9371069182389937

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidfvectorizer3, Y, test_size = 0.3, random_state = 265)


t3 = CV.fit(X_train, y_train)
t3_predict = CV.predict(X_test)
T3_Accuracy = t3.score(X_test,y_test)
T3_Accuracy

0.9308176100628931

In [21]:
#3.c.
result = pd.DataFrame()
result['type'] = ['CV_11','CV_12','CV_13','TV_11','TV_12','TV_13']
result['accuracy'] = [C1_Accuracy, C2_Accuracy, C3_Accuracy, T1_Accuracy, T2_Accuracy, T3_Accuracy]
result

,type,accuracy
0,CV_11,0.915094
1,CV_12,0.927673
2,CV_13,0.911950
3,TV_11,0.940252
4,TV_12,0.937107
5,TV_13,0.930818


In [ ]:
#4.a
#The traditional Newton method requires the inverse of The Hessian matrix every iteration,
#which is very complicated. In order to avoid the inverse of the matrix, Newton-CG uses CG conjugate
#gradient method to solve the linear equations, thus avoiding the inverse of the matrix.
#Newton-cg iteratively optimizes the loss function by using the hessian matrix, the second derivative
#matrix of the loss function. Therefore, it cannot be used for L1 regularization without continuous derivatives,
#but only for L2 regularization. And it is often used as we have a large sample. 

In [ ]:
#4.b
#It also, uses the second derivative matrix (Hessian Matrix) of the loss function to optimize the loss
#function iteratively. In BFGS algorithm, each iteration requires the Hesse matrix obtained by the previous
#iteration. The storage space of the matrix is at least N(N+1)/2, and N is the characteristic dimension.
#For high-dimensional application scenarios, the storage space required will be very large. The basic idea
#of L-BFGS is to replace the previous Hesse matrix by storing a small amount of data from the previous
#m iterations. L-bfgs algorithm can be understood as another approximation of BFGS algorithm, that reduce storage. 

In [ ]:
#4.c
#LIBLINEAR is a linear classifier for data with millions of instances and features.
#Liblinear is implemented by the open source Liblinear library, and the internal use of the axis descent method
#to iteratively optimize the loss function. It supports L2-regularized logistic regression (LR), L2-loss and L1-loss
#linear support vector machines. It has advantages like simple usage, rich documentation, and open source license

In [ ]:
#4.d
#Sag, also known as stochastic mean gradient descent, is a variant of gradient descent method.
#The difference between sag and ordinary gradient descent method is that only a part of samples are used
#in each iteration to calculate the gradient, which is suitable for a large number of sample data.
#In each calculation, the value of two gradients is used, one is the gradient value of the previous iteration
#and the other is the new gradient value. The SAG algorithm maintains an old gradient YI in memory for each
#sample, randomly selects a sample I to update D, and uses D to update parameter X. Specifically, the updated
#term d comes from replacing the old gradient y_i in d with a new gradient fi'(x), which is what d = d-yi + fi'(x)
#means. Thus, each update only computes the gradient of one sample, not the gradient of all samples.
#The computational overhead is the same as SGD, but the memory overhead is much higher.

In [ ]:
#4.e
#The SAGA algorithm is an accelerated version of the SAG algorithm. Like SAG, it neither operates within a
#loop nor calculates batch gradients (except at initial points). In each iteration, it calculates random vectors
#as the average of random gradients from previous iterations. In particular, for k iterations, the algorithm will
#store [formula], for all previous [formulas], where [formula] represents the most recent iteration computed by
#[formula]. The integers [formula] are chosen arbitrarily.